In [70]:
import kagglehub
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import pandas as pd
from networkx.algorithms.community import louvain_communities
from collections import Counter
import re
from collections import defaultdict


# Download latest version
path = kagglehub.dataset_download("harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows")
print("Path to dataset files:", path)

df = pd.read_csv(f"{path}/imdb_top_1000.csv")
# print(df[:2])
print(df.info())

Path to dataset files: /Users/karenstentoft/.cache/kagglehub/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows/versions/1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 1

In [71]:
df = df.drop(columns=["Poster_Link", "Meta_score", "Certificate"])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   1000 non-null   object 
 1   Released_Year  1000 non-null   object 
 2   Runtime        1000 non-null   object 
 3   Genre          1000 non-null   object 
 4   IMDB_Rating    1000 non-null   float64
 5   Overview       1000 non-null   object 
 6   Director       1000 non-null   object 
 7   Star1          1000 non-null   object 
 8   Star2          1000 non-null   object 
 9   Star3          1000 non-null   object 
 10  Star4          1000 non-null   object 
 11  No_of_Votes    1000 non-null   int64  
 12  Gross          831 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 101.7+ KB
None


In [3]:
# df = df.dropna().reset_index(drop=True)
# print(df['Gross'])

0       28,341,469
1      134,966,411
2      534,858,444
3       57,300,000
4        4,360,000
          ...     
826        696,690
827      1,378,435
828    141,843,612
829     13,780,024
830     30,500,000
Name: Gross, Length: 831, dtype: object


# Add imdb id

In [76]:
import requests

# API_KEY = "2aaf18d5"  
API_KEY = "d8f1a797"  
url = f"http://www.omdbapi.com/?apikey={API_KEY}&t=Inception"



def get_movie_info(title):
    url = "http://www.omdbapi.com/"
    params = {
        "t": title,
        "apikey": API_KEY
    }
    r = requests.get(url, params=params)
    return r.json()

In [77]:
df["imdbId"] = None

In [78]:
for idx, row in df.iterrows():
    title = row["Series_Title"]
    info = get_movie_info(title)
    if info.get("Response") == "True":
        df.at[idx, "imdbId"] = info.get("imdbID")

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   1000 non-null   object 
 1   Released_Year  1000 non-null   object 
 2   Runtime        1000 non-null   object 
 3   Genre          1000 non-null   object 
 4   IMDB_Rating    1000 non-null   float64
 5   Overview       1000 non-null   object 
 6   Director       1000 non-null   object 
 7   Star1          1000 non-null   object 
 8   Star2          1000 non-null   object 
 9   Star3          1000 non-null   object 
 10  Star4          1000 non-null   object 
 11  No_of_Votes    1000 non-null   int64  
 12  Gross          831 non-null    object 
 13  imdbId         867 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 109.5+ KB
None


In [81]:
df_raw = df.copy()

In [82]:
df = df_raw.dropna(subset = ["imdbId"])

In [84]:
df["imdbId"] = (
    df["imdbId"].str.replace("tt", "")
    .str.lstrip("0")
    .astype(int)
)

/var/folders/_w/g2kh2v8d5yxb4h0fkd7pblpm0000gn/T/ipykernel_37724/2553034127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["imdbId"] = (


In [88]:
df_links = pd.read_csv("links.csv")

df_links = df_links.drop(columns=["tmdbId"])

print(df_links)

       movieId    imdbId
0            1    114709
1            2    113497
2            3    113228
3            4    114885
4            5    113041
...        ...       ...
86532   288967  14418234
86533   288971  11162178
86534   288975     70199
86535   288977  23050520
86536   288983  11644948

[86537 rows x 2 columns]


In [89]:
df_combined = df.merge(df_links, on="imdbId")

print(df_combined.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791 entries, 0 to 790
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   791 non-null    object 
 1   Released_Year  791 non-null    object 
 2   Runtime        791 non-null    object 
 3   Genre          791 non-null    object 
 4   IMDB_Rating    791 non-null    float64
 5   Overview       791 non-null    object 
 6   Director       791 non-null    object 
 7   Star1          791 non-null    object 
 8   Star2          791 non-null    object 
 9   Star3          791 non-null    object 
 10  Star4          791 non-null    object 
 11  No_of_Votes    791 non-null    int64  
 12  Gross          680 non-null    object 
 13  imdbId         791 non-null    int64  
 14  movieId        791 non-null    int64  
dtypes: float64(1), int64(3), object(11)
memory usage: 92.8+ KB
None


In [92]:
df_tags = pd.read_csv("tags.csv")

df_tags = df_tags.drop(columns= ["timestamp"])

df_tags_grouped = df_tags.groupby("movieId").agg(lambda x: list(x))
df_tags_grouped = df_tags_grouped.rename(columns={"userId": "userId_tags"})

In [95]:
df_combined2 = df_combined.merge(df_tags_grouped, on="movieId")

In [96]:
df_ratings = pd.read_csv("ratings.csv")
df_ratings = df_ratings.drop(columns= ["timestamp"])


df_ratings_grouped = df_ratings.groupby("movieId").agg(lambda x: list(x))
df_ratings_grouped = df_ratings_grouped.rename(columns={"userId": "userId_ratings"})


                                            userId_ratings  \
movieId                                                      
1        [1, 2, 7, 10, 12, 14, 21, 24, 32, 35, 42, 44, ...   
2        [2, 9, 14, 21, 39, 41, 50, 51, 72, 73, 79, 82,...   
3        [7, 9, 41, 51, 73, 88, 149, 177, 190, 196, 200...   
4        [41, 260, 347, 670, 934, 1408, 1677, 1678, 174...   
5        [9, 41, 56, 72, 73, 88, 135, 149, 160, 189, 19...   

                                                    rating  
movieId                                                     
1        [4.0, 5.0, 4.0, 3.0, 5.0, 2.5, 3.0, 4.5, 3.5, ...  
2        [3.0, 4.0, 2.5, 3.0, 1.5, 4.0, 2.5, 4.0, 2.5, ...  
3        [3.0, 5.0, 4.0, 3.0, 1.5, 3.0, 0.5, 3.0, 2.5, ...  
4        [3.0, 5.0, 2.0, 3.0, 4.0, 3.0, 2.0, 3.0, 3.0, ...  
5        [3.0, 3.0, 2.0, 3.5, 3.5, 4.0, 3.0, 2.5, 3.0, ...  


In [123]:
df_final = df_combined2.merge(df_ratings_grouped, on="movieId")

print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Series_Title    784 non-null    object 
 1   Released_Year   784 non-null    object 
 2   Runtime         784 non-null    object 
 3   Genre           784 non-null    object 
 4   IMDB_Rating     784 non-null    float64
 5   Overview        784 non-null    object 
 6   Director        784 non-null    object 
 7   Star1           784 non-null    object 
 8   Star2           784 non-null    object 
 9   Star3           784 non-null    object 
 10  Star4           784 non-null    object 
 11  No_of_Votes     784 non-null    int64  
 12  Gross           677 non-null    object 
 13  imdbId          784 non-null    int64  
 14  movieId         784 non-null    int64  
 15  userId_tags     784 non-null    object 
 16  tag             784 non-null    object 
 17  userId_ratings  784 non-null    obj

In [125]:
df_final = df_final.rename(columns={
    "Series_Title": "Title",
    "IMDB_Rating": "imdb_rating",
    "Overview": "Description",
    "No_of_Votes": "imdb_no_of_votes",
    "imdbId": "imdb_id",
    "movieId": "gl_id",
    "userId_tags": "gl_user_id_tags",
    "tag": "gl_tag",
    "userId_ratings": "gl_user_id_ratings",
    "rating": "gl_rating"
})

In [126]:
df_final['Released_Year'] = (
    df_final['Released_Year']
    .astype(str)               # sikrer str
    .str.extract(r'(\d{4})')   # trækker første 4 cifre ud
    [0]                        # extract returnerer DataFrame → tag første kolonne
    .astype(float)             # konverter til float (så NaN tillades)
)


In [129]:
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               784 non-null    object 
 1   Released_Year       783 non-null    float64
 2   Runtime             784 non-null    object 
 3   Genre               784 non-null    object 
 4   imdb_rating         784 non-null    float64
 5   Description         784 non-null    object 
 6   Director            784 non-null    object 
 7   Star1               784 non-null    object 
 8   Star2               784 non-null    object 
 9   Star3               784 non-null    object 
 10  Star4               784 non-null    object 
 11  imdb_no_of_votes    784 non-null    int64  
 12  Gross               677 non-null    object 
 13  imdb_id             784 non-null    int64  
 14  gl_id               784 non-null    int64  
 15  gl_user_id_tags     784 non-null    object 
 16  gl_tag  

In [132]:
df_final["gl_user_id_tags"] = df_final["gl_user_id_tags"].apply(lambda lst: list(set(lst)))

In [135]:
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               784 non-null    object 
 1   Released_Year       783 non-null    float64
 2   Runtime             784 non-null    object 
 3   Genre               784 non-null    object 
 4   imdb_rating         784 non-null    float64
 5   Description         784 non-null    object 
 6   Director            784 non-null    object 
 7   Star1               784 non-null    object 
 8   Star2               784 non-null    object 
 9   Star3               784 non-null    object 
 10  Star4               784 non-null    object 
 11  imdb_no_of_votes    784 non-null    int64  
 12  Gross               677 non-null    object 
 13  imdb_id             784 non-null    int64  
 14  gl_id               784 non-null    int64  
 15  gl_user_id_tags     784 non-null    object 
 16  gl_tag  

In [143]:
df_final

,Title,Released_Year,Runtime,Genre,imdb_rating,Description,Director,Star1,Star2,Star3,Star4,imdb_no_of_votes,Gross,imdb_id,gl_id,gl_user_id_tags,gl_tag,gl_user_id_ratings,gl_rating
0,The Shawshank Redemption,1994.0,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",111161,318,"[73742, 229398, 319521, 270371, 327715, 180262...","[in, classic, drama, great acting, hope, inspi...","[2, 3, 4, 5, 6, 9, 11, 14, 15, 22, 26, 27, 30,...","[5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 3.5, 3.5, ..."
1,The Godfather,1972.0,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",68646,858,"[229382, 231434, 272396, 274445, 88080, 116752...","[Mafia, too long, boring, atmospheric, Mafia, ...","[3, 4, 11, 12, 13, 15, 26, 30, 31, 33, 35, 40,...","[5.0, 3.0, 4.0, 3.0, 4.0, 5.0, 5.0, 5.0, 3.0, ..."
2,The Dark Knight,2008.0,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",468569,58559,"[116739, 100356, 249860, 92166, 83976, 180232,...","[Atmospheric, Batman, comic book, dark, Heath ...","[6, 10, 11, 14, 15, 17, 22, 30, 37, 40, 44, 49...","[5.0, 5.0, 4.5, 5.0, 3.0, 5.0, 4.0, 5.0, 5.0, ..."
3,The Godfather: Part II,1974.0,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",71562,1221,"[158208, 275458, 164360, 22026, 261643, 14, 17...","[Al Pacino, mafia, classic, complex characters...","[15, 26, 30, 35, 53, 57, 78, 86, 99, 104, 105,...","[5.0, 5.0, 4.5, 4.5, 4.5, 5.0, 5.0, 4.5, 5.0, ..."
4,12 Angry Men,1957.0,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",50083,1203,"[231434, 77839, 231441, 20505, 112668, 151600,...","[group psychology, classic, courtroom, courtro...","[30, 31, 33, 44, 51, 76, 78, 96, 105, 112, 123...","[5.0, 2.0, 5.0, 5.0, 5.0, 4.5, 5.0, 0.5, 5.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,Breakfast at Tiffany's,1961.0,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN,54698,902,"[222223, 315416, 205338, 46623, 185887, 307234...","[Audrey Hepburn, Netflix Finland, 1960s, apart...","[48, 99, 137, 148, 149, 175, 181, 189, 203, 22...","[5.0, 5.0, 4.0, 2.0, 3.5, 5.0, 2.5, 3.5, 4.0, ..."
780,Giant,1956.0,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN,49261,948,"[8833, 83717, 270499, 7716, 215490, 241095, 23...","[National Film Registry, on dvr, epic, Classic...","[148, 481, 490, 548, 927, 1678, 1767, 1783, 18...","[3.0, 3.0, 4.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.5, ..."
781,From Here to Eternity,1953.0,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000",45793,1944,"[238092, 222223, 126357, 210197, 226209, 7716,...","[National Film Registry, Nana viu, Deborah Ker...","[94, 149, 442, 444, 481, 487, 525, 548, 562, 6...","[4.5, 2.5, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0, ..."
782,Lifeboat,1944.0,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN,37017,2202,"[88449, 172802, 190434, 108132, 215490, 229729...","[reviewed, Hitchcock, holes40s, tivo, history,...","[105, 149, 301, 663, 927, 1057, 1156, 1306, 13...","[5.0, 3.0, 4.0, 3.5, 4.0, 4.5, 4.0, 4.0, 3.0, ..."


In [145]:
# Creating the graph
G = nx.Graph()

for i in range(len(df_final)):
    G.add_node(df_final['Title'][i],
    release_year = int(df_final['Released_Year'][i]) if not pd.isna(df_final['Released_Year'][i]) else None,
    runtime = int((df_final['Runtime'][i]).replace(' min', '')),
    genre = (df_final['Genre'][i]).replace(" ", "").split(','),
    imdb_rating = float(df_final['imdb_rating'][i]),
    description = df_final['Description'][i],
    director = df_final['Director'][i],
    stars = [df_final['Star1'][i], df_final['Star2'][i], df_final['Star3'][i], df_final['Star4'][i]],
    imdb_no_of_votes = int(df_final['imdb_no_of_votes'][i]),
    gross = int((df_final['Gross'][i]).replace(',', '')) if not pd.isna(df_final['Gross'][i]) else None,
    imdb_id = df_final["imdb_id"][i],
    gl_id = df_final["gl_id"][i],
    gl_user_id_tags = df_final["gl_user_id_tags"][i],
    gl_tag = df_final["gl_tag"][i],
    gl_user_id_ratings = df_final["gl_user_id_ratings"][i],
    gl_rating = df_final["gl_rating"][i]
    )

print(G)



Graph with 783 nodes and 0 edges
